## validate ingredient/recipes:

In [2]:
import os
import yaml
import datetime

import ipywidgets as widgets
from IPython.display import display

from src.Settings import RECIPE_DIR
from src.Recipe import list_recipes, get_ingredient_unit, Recipe

def format_ingredient_name(ingredient_name):
    ingredient_name = ingredient_name.lower()
    ingredient_name = ingredient_name.replace(" ", "")
    
    if ingredient_name.endswith("ies"):
        ingredient_name = ingredient_name[:-3] + "y"
    if ingredient_name.endswith("es"):
        ingredient_name = ingredient_name[:-2]
    if ingredient_name.endswith("s"):
        ingredient_name = ingredient_name[:-1]
        
    return ingredient_name

ingredient_list = []
with open(os.path.join(os.getcwd(),"rsrc", "ingredients.yaml"), 'r') as ingf:
    ingredients = yaml.load(ingf, Loader=yaml.FullLoader)
    for k, v in ingredients.items():
        ingredient_list.append(format_ingredient_name(k))
        ingredient_list += [format_ingredient_name(alt) for alt in v.get("alternatives", [])]

recipe_list = list_recipes()
for recipe_name, recipe_data in recipe_list.items():
    recipe = Recipe.load(recipe_data["filename"], recipe_data["id"])
    for ingredient in recipe.ingredients:
        ingredient_name = format_ingredient_name(ingredient["name"])
        
        if not ingredient_name in ingredient_list:
            print(f"{recipe_name}:  {ingredient_name}")
            print(f"{recipe_data}")
            break

checkboxes_list = [widgets.Label(value="Months: ")]
for i in range(1, 13):
    month_name = datetime.datetime.strptime(str(i), "%m").strftime("%B")
    checkboxes_list.append(
        widgets.Checkbox(
            value=False,
            description=month_name,
            disabled=False,
            indent=False
        )
    )

textbox = widgets.Text(value='', description='New ingredient', disabled=False)
display(textbox)
alternative_list = widgets.VBox([])
display(alternative_list)

def add_alt(b=None):
    alternative_list.children += (widgets.Text(value='', description='aternative', disabled=False),)
def remove_alt(b=None):
    alternative_list.children = alternative_list.children[:-1]
    
add_button = widgets.Button(
                    disabled=False,
                    button_style='',
                    tooltip='add',
                    icon='plus',
                )
remove_button = widgets.Button(
                    disabled=False,
                    button_style='',
                    tooltip='add',
                    icon='minus',
                )
add_button.on_click(add_alt)
remove_button.on_click(remove_alt)
display(add_button)
display(remove_button)

checkboxes = widgets.VBox(checkboxes_list)
display(checkboxes)

def save(b=None):

    content = {}
    if len(alternative_list.children) > 0:
        content["alternatives"] = []
        for child in alternative_list.children:
            content["alternatives"].append(child.value)
    seasons = []
    for i, checkbox in enumerate(checkboxes.children[1:]):
        if checkbox.value:
            seasons.append(i+1)
    if seasons:
        content["season"] = seasons
    ingredients = {}
    with open(os.path.join(os.getcwd(),"rsrc", "ingredients.yaml"), 'r') as ingf:
        ingredients = yaml.load(ingf, Loader=yaml.FullLoader)
    ingredients[textbox.value] = content
    with open(os.path.join(os.getcwd(),"rsrc", "ingredients.yaml"), 'w') as ingf:
        yaml.dump(ingredients, ingf, default_flow_style=False)

save_button = widgets.Button(
    disabled=False,
    button_style='',
    tooltip='Save',
    icon='play',
)
save_button.on_click(save)
display(save_button)

Text(value='', description='New ingredient')

VBox()

Button(icon='plus', style=ButtonStyle(), tooltip='add')

Button(icon='minus', style=ButtonStyle(), tooltip='add')

Button(icon='play', style=ButtonStyle(), tooltip='Save')